# Lithops Moments in Time dataset example
## Video/image prediction
In this notebook we will process video clips from the MiT dataset at scale with Lithops
by predicting the actions with a pretrained ResNet50 model and then counting how many
occurrences of each category have been predicted.


In [3]:
import time
import builtins
import torch.optim
import torch.nn.parallel
from torch import save, load
from torch.nn import functional as F

from utils import extract_frames
from models import load_model, load_transform, load_categories

from lithops.multiprocessing import Pool, Queue
from lithops.multiprocessing.util import get_uuid

### Backends
The same program can be run in a local environtment with processes or executed by
functions in the cloud. After we choose a backend, only a few file locations must
be changed. In this example we will be using the cloud functions backend.

We will be using a custom runtime for our functions which has torch, torchvision,
ffmpeg and opencv-python modules already installed.
We will store the pretrained weights in the cloud so that functions can access it.
Then, after functions get the models weights they will start preprocessing input
videos and inferring them one by one.
  
Later in this notebook, we will see a little improvement detail to this process.  


In [4]:
LOCAL_EXEC = False

In [5]:
INPUT_DATA_DIR = 'momentsintime/input_data'

if LOCAL_EXEC:
    import os
    from builtins import open
    pool_initargs = {
        'compute_backend': 'localhost',
        'storage_backend': 'localhost'
        }
    weights_location = '/dev/shm/model_weights'
    INPUT_DATA_DIR = os.path.abspath(INPUT_DATA_DIR)

else:
    from lithops.storage.cloud_proxy import os, open
    pool_initargs = {
        'compute_backend': 'ibm_cf',
        'storage_backend': 'ibm_cos',
        'runtime': 'dhak/pywren-runtime-pytorch:3.6',
        'runtime_memory': 2048
        }
    weights_location = 'momentsintime/models/model_weights'
    

In [6]:
video_locations = [os.path.join(INPUT_DATA_DIR, name) for name in os.listdir(INPUT_DATA_DIR)]

As you can see, we have masked the `open` function and `os` module with a proxy
to manage files from the cloud transparently.  
We will use `builtins.open` from now on to explicitly access a local file as some accesses have to occur in the very same machine.

### Download pretrained ResNet50 model weights and save them in a directory accessible by all functions (`weights_location`)

In [11]:
ROOT_URL = 'http://moments.csail.mit.edu/moments_models'
WEIGHTS_FILE = 'moments_RGB_resnet50_imagenetpretrained.pth.tar'

if not os.access(WEIGHTS_FILE, os.R_OK):
    os.system('wget ' + '/'.join([ROOT_URL, WEIGHTS_FILE]))

with builtins.open(WEIGHTS_FILE, 'rb') as f_in:
    weights = f_in.read()
with open(weights_location, 'wb') as f_out:
    f_out.write(weights)

--2021-09-18 21:43:16--  http://moments.csail.mit.edu/moments_models/moments_RGB_resnet50_imagenetpretrained.pth.tar
Resolving moments.csail.mit.edu (moments.csail.mit.edu)... 128.30.100.222
Connecting to moments.csail.mit.edu (moments.csail.mit.edu)|128.30.100.222|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 193898685 (185M) [application/x-tar]
Saving to: ‘moments_RGB_resnet50_imagenetpretrained.pth.tar’

     0K .......... .......... .......... .......... ..........  0%  239K 13m11s
    50K .......... .......... .......... .......... ..........  0%  550K 9m27s
   100K .......... .......... .......... .......... ..........  0% 1,44M 7m1s
   150K .......... .......... .......... .......... ..........  0% 31,5M 5m17s
   200K .......... .......... .......... .......... ..........  0%  601K 5m17s
   250K .......... .......... .......... .......... ..........  0% 2,14M 4m38s
   300K .......... .......... .......... .......... ..........  0% 5,52M 4m3s
   350K ..

  6050K .......... .......... .......... .......... ..........  3%  220K 40s
  6100K .......... .......... .......... .......... ..........  3% 33,8M 40s
  6150K .......... .......... .......... .......... ..........  3% 29,9M 39s
  6200K .......... .......... .......... .......... ..........  3%  122K 51s
  6250K .......... .......... .......... .......... ..........  3% 28,3M 51s
  6300K .......... .......... .......... .......... ..........  3% 62,5M 50s
  6350K .......... .......... .......... .......... ..........  3%  478K 53s
  6400K .......... .......... .......... .......... ..........  3% 20,4M 53s
  6450K .......... .......... .......... .......... ..........  3% 10,3M 52s
  6500K .......... .......... .......... .......... ..........  3%  513K 55s
  6550K .......... .......... .......... .......... ..........  3% 58,0M 54s
  6600K .......... .......... .......... .......... ..........  3% 14,7M 54s
  6650K .......... .......... .......... .......... ..........  3% 10,1M 54s

 12450K .......... .......... .......... .......... ..........  6% 6,20M 43s
 12500K .......... .......... .......... .......... ..........  6% 46,7M 43s
 12550K .......... .......... .......... .......... ..........  6% 13,1M 43s
 12600K .......... .......... .......... .......... ..........  6% 11,1M 43s
 12650K .......... .......... .......... .......... ..........  6% 14,8M 43s
 12700K .......... .......... .......... .......... ..........  6% 1,13M 43s
 12750K .......... .......... .......... .......... ..........  6% 5,55M 43s
 12800K .......... .......... .......... .......... ..........  6% 11,5M 43s
 12850K .......... .......... .......... .......... ..........  6% 15,1M 43s
 12900K .......... .......... .......... .......... ..........  6% 5,70M 43s
 12950K .......... .......... .......... .......... ..........  6% 17,8M 43s
 13000K .......... .......... .......... .......... ..........  6% 11,2M 43s
 13050K .......... .......... .......... .......... ..........  6% 10,3M 43s

 18100K .......... .......... .......... .......... ..........  9% 1,24M 39s
 18150K .......... .......... .......... .......... ..........  9% 3,15M 39s
 18200K .......... .......... .......... .......... ..........  9% 12,2M 39s
 18250K .......... .......... .......... .......... ..........  9% 17,1M 39s
 18300K .......... .......... .......... .......... ..........  9% 13,0M 39s
 18350K .......... .......... .......... .......... ..........  9% 10,4M 39s
 18400K .......... .......... .......... .......... ..........  9% 13,8M 38s
 18450K .......... .......... .......... .......... ..........  9% 8,38M 38s
 18500K .......... .......... .......... .......... ..........  9% 6,97M 38s
 18550K .......... .......... .......... .......... ..........  9% 51,2M 38s
 18600K .......... .......... .......... .......... ..........  9% 2,69M 38s
 18650K .......... .......... .......... .......... ..........  9% 1,32M 39s
 18700K .......... .......... .......... .......... ..........  9% 2,87M 39s

 24400K .......... .......... .......... .......... .......... 12%  794K 36s
 24450K .......... .......... .......... .......... .......... 12% 26,9M 36s
 24500K .......... .......... .......... .......... .......... 12% 8,23M 36s
 24550K .......... .......... .......... .......... .......... 12% 10,2M 36s
 24600K .......... .......... .......... .......... .......... 13% 44,1M 36s
 24650K .......... .......... .......... .......... .......... 13% 10,4M 36s
 24700K .......... .......... .......... .......... .......... 13% 13,7M 36s
 24750K .......... .......... .......... .......... .......... 13% 8,75M 36s
 24800K .......... .......... .......... .......... .......... 13% 4,40M 36s
 24850K .......... .......... .......... .......... .......... 13% 66,1M 36s
 24900K .......... .......... .......... .......... .......... 13% 9,84M 36s
 24950K .......... .......... .......... .......... .......... 13% 13,9M 36s
 25000K .......... .......... .......... .......... .......... 13%  491K 36s

...

162050K .......... .......... .......... .......... .......... 85% 10,1M 7s
162100K .......... .......... .......... .......... .......... 85% 9,94M 7s
162150K .......... .......... .......... .......... .......... 85% 1,24M 7s
162200K .......... .......... .......... .......... .......... 85% 20,6M 7s
162250K .......... .......... .......... .......... .......... 85% 1,28M 7s
162300K .......... .......... .......... .......... .......... 85% 3,26M 7s
162350K .......... .......... .......... .......... .......... 85% 13,9M 7s
162400K .......... .......... .......... .......... .......... 85% 10,2M 7s
162450K .......... .......... .......... .......... .......... 85% 1,12M 7s
162500K .......... .......... .......... .......... .......... 85% 55,9M 7s
162550K .......... .......... .......... .......... .......... 85% 21,4M 7s
162600K .......... .......... .......... .......... .......... 85% 1006K 7s
162650K .......... .......... .......... .......... .......... 85% 24,9M 7s
162700K ....

167650K .......... .......... .......... .......... .......... 88% 1,89M 6s
167700K .......... .......... .......... .......... .......... 88%  845K 6s
167750K .......... .......... .......... .......... .......... 88% 24,8M 6s
167800K .......... .......... .......... .......... .......... 88% 12,6M 6s
167850K .......... .......... .......... .......... .......... 88% 11,6M 6s
167900K .......... .......... .......... .......... .......... 88% 4,83M 6s
167950K .......... .......... .......... .......... .......... 88% 10,1M 6s
168000K .......... .......... .......... .......... .......... 88%  866K 6s
168050K .......... .......... .......... .......... .......... 88% 1,99M 5s
168100K .......... .......... .......... .......... .......... 88% 19,5M 5s
168150K .......... .......... .......... .......... .......... 88% 10,9M 5s
168200K .......... .......... .......... .......... .......... 88% 9,84M 5s
168250K .......... .......... .......... .......... .......... 88% 8,03M 5s
168300K ....

173450K .......... .......... .......... .......... .......... 91% 9,41M 4s
173500K .......... .......... .......... .......... .......... 91% 10,0M 4s
173550K .......... .......... .......... .......... .......... 91%  614K 4s
173600K .......... .......... .......... .......... .......... 91% 53,9M 4s
173650K .......... .......... .......... .......... .......... 91% 4,63M 4s
173700K .......... .......... .......... .......... .......... 91% 69,8M 4s
173750K .......... .......... .......... .......... .......... 91% 9,34M 4s
173800K .......... .......... .......... .......... .......... 91% 14,9M 4s
173850K .......... .......... .......... .......... .......... 91% 1,59M 4s
173900K .......... .......... .......... .......... .......... 91%  921K 4s
173950K .......... .......... .......... .......... .......... 91% 63,7M 4s
174000K .......... .......... .......... .......... .......... 91% 4,38M 4s
174050K .......... .......... .......... .......... .......... 91% 9,93M 4s
174100K ....

179200K .......... .......... .......... .......... .......... 94% 2,55M 3s
179250K .......... .......... .......... .......... .......... 94% 6,06M 3s
179300K .......... .......... .......... .......... .......... 94%  870K 3s
179350K .......... .......... .......... .......... .......... 94% 14,7M 3s
179400K .......... .......... .......... .......... .......... 94% 1,01M 3s
179450K .......... .......... .......... .......... .......... 94% 5,97M 3s
179500K .......... .......... .......... .......... .......... 94% 1,15M 3s
179550K .......... .......... .......... .......... .......... 94%  104M 3s
179600K .......... .......... .......... .......... .......... 94% 11,3M 3s
179650K .......... .......... .......... .......... .......... 94% 6,17M 3s
179700K .......... .......... .......... .......... .......... 94% 12,1M 3s
179750K .......... .......... .......... .......... .......... 94% 1,31M 3s
179800K .......... .......... .......... .......... .......... 94% 1,01M 3s
179850K ....

185100K .......... .......... .......... .......... .......... 97%  988K 1s
185150K .......... .......... .......... .......... .......... 97% 2,09M 1s
185200K .......... .......... .......... .......... .......... 97% 10,4M 1s
185250K .......... .......... .......... .......... .......... 97% 3,72M 1s
185300K .......... .......... .......... .......... .......... 97% 45,1M 1s
185350K .......... .......... .......... .......... .......... 97% 1,12M 1s
185400K .......... .......... .......... .......... .......... 97% 39,1M 1s
185450K .......... .......... .......... .......... .......... 97% 13,3M 1s
185500K .......... .......... .......... .......... .......... 97% 1,50M 1s
185550K .......... .......... .......... .......... .......... 98% 2,56M 1s
185600K .......... .......... .......... .......... .......... 98% 12,6M 1s
185650K .......... .......... .......... .......... .......... 98% 14,9M 1s
185700K .......... .......... .......... .......... .......... 98% 10,3M 1s
185750K ....

### Video prediction and reduce function code


In [7]:
NUM_SEGMENTS = 16

# Get dataset categories
categories = load_categories()

# Load the video frame transform
transform = load_transform()

def predict_videos(queue, video_locations):
    with open(weights_location, 'rb') as f:
        model = load_model(f)
    model.eval()

    results = []
    local_video_loc = 'video_to_predict_{}.mp4'.format(get_uuid())

    for video_loc in video_locations:
        start = time.time()
        with open(video_loc, 'rb') as f_in:
            with builtins.open(local_video_loc, 'wb') as f_out:
                f_out.write(f_in.read())

        # Obtain video frames
        frames = extract_frames(local_video_loc, NUM_SEGMENTS)

        # Prepare input tensor [num_frames, 3, 224, 224]
        input_v = torch.stack([transform(frame) for frame in frames])

        # Make video prediction
        with torch.no_grad():
            logits = model(input_v)
            h_x = F.softmax(logits, 1).mean(dim=0)
            probs, idx = h_x.sort(0, True)

        # Output the prediction
        result = dict(key=video_loc)
        result['prediction'] = (idx[0], round(float(probs[0]), 5))
        result['iter_duration'] = time.time() - start
        results.append(result)
    queue.put(results)

# Counts how many predictions of each category have been made
def reduce(queue, n):
    pred_x_categ = {}
    for categ in categories:
        pred_x_categ[categ] = 0

    checkpoint = 0.2
    res_count = 0

    for i in range(n):
        results = queue.get()
        res_count += len(results)
        for res in results:
            idx, prob = res['prediction']
            pred_x_categ[categories[idx]] += 1

        # print progress
        if i >= (N * checkpoint):
            print('Processed {} results.'.format(res_count))
            checkpoint += 0.2

    return pred_x_categ

### Map functions
Similar to the `multiprocessing` module API, we use a Pool to map the video keys
across n workers (concurrency). However, we do not have to instantiate a Pool of
n workers *specificly*, it is the map function that will invoke as many workers according
to the length of the list.

In [8]:
CONCURRENCY = 100

In [14]:
queue = Queue()
pool = Pool(initargs=pool_initargs)

# Slice data keys
N = min(CONCURRENCY, len(video_locations))
iterable = [(queue, video_locations[n::CONCURRENCY]) 
            for n in range(N)]

# Map and reduce on the go
start = time.time()
pool.map_async(func=predict_videos, iterable=iterable)
pred_x_categ = reduce(queue, N)
end = time.time()
    
print('\nDone.')
print('Videos processed:', len(video_locations))
print('Total duration:', round(end - start, 2), 'sec\n')

for categ, count in pred_x_categ.items():
    if count != 0:
        print('{}: {}'.format(categ, count))

Exception: Redis section not found in you config

---------------

## Performance improvement
Now, since we know every function will have to pull the model weights from
the cloud storage, we can actually pack these weights with the runtime image
and reduce the start-up cost substantially.

In [9]:
pool_initargs['runtime'] = 'dhak/pywren-runtime-resnet'
weights_location = '/momentsintime/model_weights'

In [10]:
def predict_videos(queue, video_locations):
    # force local file access on new weights_location
    with builtins.open(weights_location, 'rb') as f:
        model = load_model(f)
    model.eval()

    results = []
    local_video_loc = 'video_to_predict_{}.mp4'.format(get_uuid())

    for video_loc in video_locations:
        start = time.time()
        with open(video_loc, 'rb') as f_in:
            with builtins.open(local_video_loc, 'wb') as f_out:
                f_out.write(f_in.read())

        # Obtain video frames
        frames = extract_frames(local_video_loc, NUM_SEGMENTS)

        # Prepare input tensor [num_frames, 3, 224, 224]
        input_v = torch.stack([transform(frame) for frame in frames])

        # Make video prediction
        with torch.no_grad():
            logits = model(input_v)
            h_x = F.softmax(logits, 1).mean(dim=0)
            probs, idx = h_x.sort(0, True)

        # Output the prediction
        result = dict(key=video_loc)
        result['prediction'] = (idx[0], round(float(probs[0]), 5))
        result['iter_duration'] = time.time() - start
        results.append(result)
    queue.put(results)

In [11]:
queue = Queue()
pool = Pool(initargs=pool_initargs)

# Slice data keys
N = min(CONCURRENCY, len(video_locations))
iterable = [(queue, video_locations[n::CONCURRENCY]) 
            for n in range(N)]

# Map and reduce on the go
start = time.time()
r = pool.map_async(func=predict_videos, iterable=iterable)
pred_x_categ = reduce(queue, N)
end = time.time()
    
print('\nDone.')
print('Videos processed:', len(video_locations))
print('Total duration:', round(end - start, 2), 'sec\n')

for categ, count in pred_x_categ.items():
    if count != 0:
        print('{}: {}'.format(categ, count))

Exception: Redis section not found in you config

### Clean

In [24]:
try:
    os.remove(weights_location)
except FileNotFoundError:
    pass

try:
    os.remove(WEIGHTS_FILE)
except FileNotFoundError:
    pass

### Dockerfiles and build scripts for both runtimes can be found in the runtime/ folder.

### Source code adapted from the demonstration in https://github.com/zhoubolei/moments_models

### Moments in Time article: http://moments.csail.mit.edu/#paper
